In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar100

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
def build_lenet_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess MNIST data
num_classes = 100
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Load the data and split it between train and test sets

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



169001437/169001437 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [ ]:
def train_lenet(model, train_data, train_labels, test_data, test_labels):
    model.save_weights(init_weights)
    model.save(init_model)
    initailsoftmax = model.predict(test_data)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        loss, accuracy = model.evaluate(test_data, test_labels, verbose=0)
        results = model.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)

        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
        if epoch ==25:
          model.save_weights(final_weights)
          model.save(final_model)
        epoch += 1




In [ ]:
model_name = 'LeNet'
dataset = 'CIFAR100'

# Creating Base Models

In [ ]:
filepath = '/content/base/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset +'_init.tf'
final_weights = filepath + model_name + dataset +'_final_weights.tf'
final_model= filepath + model_name + dataset +'_final.tf'

In [ ]:
!mkdir '/content/base/'

In [ ]:
base = build_lenet_model(input_shape, num_classes)

In [ ]:
base.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
train_lenet(base,x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 4ms/step
Epoch 1:
313/313 [==============================] - 1s 4ms/step
Test Loss: 3.3207335472106934, Test Accuracy: 0.20980000495910645
Epoch 2:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.995577096939087, Test Accuracy: 0.27059999108314514
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.8279876708984375, Test Accuracy: 0.305400013923645
Epoch 4:
313/313 [==============================] - 1s 3ms/step
Test Loss: 2.6902949810028076, Test Accuracy: 0.3352000117301941
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.649301767349243, Test Accuracy: 0.34790000319480896
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.6052699089050293, Test Accuracy: 0.358599990606308
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.6201229095458984, Test Accuracy: 0.3612000048160553
Epoch 8:
313/313 [=========================

In [ ]:
del (base)
gc.collect()
keras.backend.clear_session()

# Independant model one

In [ ]:
filepath = '/content/independant_1/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + '_final.tf'

In [ ]:
!mkdir  '/content/independant_1/'

In [ ]:
model_independant_1 = build_lenet_model(input_shape, num_classes)

In [ ]:
train_lenet(model_independant_1,x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 2s 5ms/step
Test Loss: 3.2930667400360107, Test Accuracy: 0.21060000360012054
Epoch 2:
313/313 [==============================] - 1s 3ms/step
Test Loss: 2.96954345703125, Test Accuracy: 0.2768999934196472
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.7825088500976562, Test Accuracy: 0.3133000135421753
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.642711639404297, Test Accuracy: 0.3416999876499176
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.5783371925354004, Test Accuracy: 0.3578000068664551
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.568521022796631, Test Accuracy: 0.3643999993801117
Epoch 7:
313/313 [==============================] - 1s 3ms/step
Test Loss: 2.5796239376068115, Test Accuracy: 0.36419999599456787
Epoch 8:
313/313 [==========================

In [ ]:
del (model_independant_1)
gc.collect()
keras.backend.clear_session()

# Independant 2

In [ ]:
filepath = '/content/independant_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights =filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + '_final.tf'

In [ ]:
!mkdir '/content/independant_2/'

In [ ]:
model_independant_2 = build_lenet_model(input_shape, num_classes)

In [ ]:
train_lenet(model_independant_2,x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step
Test Loss: 3.3558125495910645, Test Accuracy: 0.2003999948501587
Epoch 2:
313/313 [==============================] - 1s 2ms/step
Test Loss: 3.032379627227783, Test Accuracy: 0.260699987411499
Epoch 3:
313/313 [==============================] - 1s 3ms/step
Test Loss: 2.8380651473999023, Test Accuracy: 0.30230000615119934
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.724428415298462, Test Accuracy: 0.32910001277923584
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.6430797576904297, Test Accuracy: 0.349700003862381
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 2.6156346797943115, Test Accuracy: 0.3571999967098236
Epoch 7:
313/313 [==============================] - 1s 3ms/step
Test Loss: 2.610078811645508, Test Accuracy: 0.3643999993801117
Epoch 8:
313/313 [===========================

In [ ]:
del (model_independant_2)
gc.collect()
keras.backend.clear_session()

#Download Independant Models

In [ ]:
shutil.make_archive("base", 'zip', "base")
files.download("base.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("independant_1", 'zip', "independant_1")
files.download("independant_1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

shutil.make_archive("independant_2", 'zip', "independant_2")
files.download("independant_2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1

# TO DO EVALUATE FUNCTION


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

# Student 0.1 Alpha

In [ ]:
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)


In [ ]:
teacher_model_path = '/content/base/LeNetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)

In [ ]:
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)

In [ ]:
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

In [ ]:
!mkdir '/content/student_same/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 3.7263 - accuracy: 0.1483
Test Loss: 3.726341485977173, Test Accuracy: 0.14830000698566437
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4244 - accuracy: 0.1988
Test Loss: 3.4244258403778076, Test Accuracy: 0.1987999975681305
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2311 - accuracy: 0.2294
Test Loss: 3.231084108352661, Test Accuracy: 0.22939999401569366
Epoch 4:
313/313 [==============================] - 1s 4ms/step - loss: 3.0941 - accuracy: 0.2565
Test Loss: 3.094144821166992, Test Accuracy: 0.2565000057220459
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 3.0113 - accuracy: 0.2742
Test Loss: 3.011259078979492, Test Accuracy: 0.2741999924182892
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9332 - accuracy: 0.2910
Test Loss: 2.9331750869750977, Test Accuracy: 0.29100000858306885
Epoch 7:
313/313 [==============

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

# Student same Alpha 0.5

In [ ]:
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

In [ ]:
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

In [ ]:
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

In [ ]:
!mkdir '/content/student_same_50/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 3.7143 - accuracy: 0.1504
Test Loss: 3.7143352031707764, Test Accuracy: 0.15039999783039093
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4074 - accuracy: 0.1973
Test Loss: 3.407443046569824, Test Accuracy: 0.1973000019788742
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2210 - accuracy: 0.2326
Test Loss: 3.2210440635681152, Test Accuracy: 0.23260000348091125
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.0973 - accuracy: 0.2544
Test Loss: 3.0973057746887207, Test Accuracy: 0.25440001487731934
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 3.0025 - accuracy: 0.2741
Test Loss: 3.0024802684783936, Test Accuracy: 0.27410000562667847
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9275 - accuracy: 0.2893
Test Loss: 2.9275362491607666, Test Accuracy: 0.28929999470710754
Epoch 7:
313/313 [=========

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

# Student same Alpha 0.9

In [ ]:
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 3ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 4ms/step - loss: 3.7263 - accuracy: 0.1501
Test Loss: 3.7262701988220215, Test Accuracy: 0.1500999927520752
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4218 - accuracy: 0.1967
Test Loss: 3.4217512607574463, Test Accuracy: 0.19670000672340393
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2298 - accuracy: 0.2307
Test Loss: 3.2297685146331787, Test Accuracy: 0.23070000112056732
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.1062 - accuracy: 0.2558
Test Loss: 3.106198310852051, Test Accuracy: 0.2558000087738037
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 3.0164 - accuracy: 0.2717
Test Loss: 3.016366958618164, Test Accuracy: 0.271699994802475
Epoch 6:
313/313 [==============================] - 1s 4ms/step - loss: 2.9405 - accuracy: 0.2870
Test Loss: 2.9405133724212646, Test Accuracy: 0.28700000047683716
Epoch 7:
313/313 [=============

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

---

# Download Distill Data

In [ ]:
shutil.make_archive("student_same", 'zip', "student_same")
files.download("student_same.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("student_same_50", 'zip', "student_same_50")
files.download("student_same_50.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("student_same_90", 'zip', "student_same_90")
files.download("student_same_90.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

# Qunatisation

https://github.com/sonalimedani/TF_Quantization/blob/master/quantization.ipynb

https://medium.com/@sonalimedani/post-training-quantization-with-tensorflow-lite-on-a-keras-model-f373068966c4

**Default**: 8 Bit

In [ ]:
path = '/content/base/LeNetCIFAR100_final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 3.9201 - accuracy: 0.3268


[3.9201202392578125, 0.32679998874664307]

In [ ]:
filepath = '/content/quantised_8_bit/'
!mkdir '/content/quantised_8_bit/'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open(filepath+"8_bit.tflite", "wb").write(tflite_quant_model)

335208

In [ ]:
import os
print("Float model in Mb:", os.path.getsize(filepath+"8_bit.tflite") / float(2**20))
print("Quantized model in Mb:", os.path.getsize(filepath+"8_bit.tflite") / float(2**20))
print("Compression ratio:", os.path.getsize(filepath+"8_bit.tflite")/os.path.getsize(path))

Float model in Mb: 0.31967926025390625
Quantized model in Mb: 0.31967926025390625
Compression ratio: 81.837890625


In [ ]:
interpreter = tf.lite.Interpreter(model_path=(filepath+"8_bit.tflite"))
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_default = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_default = pd.concat([results_default, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

32.73


In [ ]:
results_default

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.212556e-10,2.747216e-05,9.251284e-03,9.435776e-04,2.133205e-03,8.117640e-05,1.147782e-06,5.882317e-06,1.657996e-04,2.266008e-05,...,3.414786e-03,8.739687e-03,5.719241e-07,2.003900e-04,6.620030e-09,1.732560e-03,1.535469e-06,7.048406e-05,1.369617e-03,1.430925e-03
1,1.716817e-07,6.883182e-06,2.292620e-03,9.283205e-04,9.515498e-03,2.481675e-06,3.927213e-05,2.230912e-06,8.257181e-08,1.920442e-05,...,7.253380e-04,1.211946e-03,3.939063e-07,1.821038e-05,4.318182e-08,1.716814e-06,2.179349e-04,2.897902e-02,2.980532e-05,2.930799e-07
2,8.424056e-13,2.783376e-08,7.458773e-04,1.155507e-04,1.156424e-01,1.368471e-11,1.247528e-07,1.407396e-09,5.335754e-07,2.748714e-08,...,1.457637e-04,1.529217e-07,2.495297e-08,1.578749e-03,6.123182e-09,3.536985e-04,5.163979e-08,5.985804e-06,7.501006e-04,1.069801e-04
3,1.699606e-06,4.486708e-07,1.301498e-04,9.030816e-06,1.117866e-03,1.666725e-04,2.481404e-07,8.140784e-10,1.336594e-08,4.326151e-07,...,5.652131e-08,6.539594e-02,1.517022e-06,3.614128e-04,3.179569e-12,3.562476e-07,4.430606e-09,1.748947e-03,3.257847e-03,1.090626e-05
4,3.769742e-05,2.460132e-05,3.051876e-08,5.871406e-07,1.029137e-08,1.172283e-06,2.363468e-11,2.206027e-07,2.352951e-08,1.010139e-06,...,5.308378e-06,1.912257e-08,9.068019e-07,1.031491e-08,7.996896e-08,1.936050e-12,7.905866e-05,2.237205e-11,1.476615e-07,6.104665e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6.204795e-04,2.067754e-05,5.056641e-09,4.051287e-04,1.255567e-06,2.680471e-12,1.369351e-03,6.510288e-06,6.881312e-08,2.755892e-05,...,4.310153e-09,1.931987e-07,1.939136e-03,1.833205e-04,1.118230e-13,1.641131e-09,4.628329e-05,3.162003e-07,6.687471e-07,8.122335e-05
9996,2.635315e-12,4.394662e-10,2.079966e-06,1.855544e-04,1.749722e-03,1.512318e-09,4.714965e-06,4.747422e-07,7.724339e-14,7.613202e-11,...,1.326261e-06,9.611949e-07,2.956578e-09,1.409621e-06,3.945510e-13,3.604594e-07,6.022253e-11,3.113291e-06,1.905921e-06,1.287697e-06
9997,3.405335e-06,9.895490e-07,6.473668e-08,9.673167e-05,2.263670e-06,2.454849e-06,2.049116e-02,5.902275e-04,2.728263e-07,5.691364e-07,...,1.051002e-03,4.222670e-04,3.097592e-02,1.216022e-05,2.192609e-11,1.764374e-07,7.052624e-05,1.430276e-03,5.663482e-06,3.716036e-02
9998,1.327816e-07,1.422173e-10,5.111753e-05,3.335210e-03,1.915008e-03,2.298381e-05,1.786811e-06,1.229453e-06,2.025586e-04,4.744878e-06,...,1.443788e-04,7.492131e-05,3.447751e-06,1.138174e-04,2.560385e-08,3.943873e-10,3.920659e-07,7.369397e-01,4.198915e-05,7.354130e-03


In [ ]:
results_default.to_csv((filepath+"25_softmax.csv"),index=False)

**16 bit**

In [ ]:
model = tf.keras.models.load_model(path)


In [ ]:
filepath = '/content/quantised_16_bit/'
!mkdir '/content/quantised_16_bit/'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open((filepath +"16_bit.tflite"), "wb").write(tflite_quant_model)

659204

In [ ]:
import os
print("Float model in Mb:", os.path.getsize(filepath +"16_bit.tflite") / float(2**20))
print("Quantized model in Mb:", os.path.getsize(filepath +"16_bit.tflite") / float(2**20))
print("Compression ratio:", os.path.getsize(filepath +"16_bit.tflite")/os.path.getsize(path))

Float model in Mb: 0.6286659240722656
Quantized model in Mb: 0.6286659240722656
Compression ratio: 160.9384765625


In [ ]:
interpreter = tf.lite.Interpreter(model_path=filepath +"16_bit.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_16_b = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_16_b = pd.concat([results_16_b, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

32.68


In [ ]:
results_16_b

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.154083e-10,2.433531e-05,7.738088e-03,9.048959e-04,2.049274e-03,8.322712e-05,1.297938e-06,4.698501e-06,2.371589e-04,1.635938e-05,...,3.856127e-03,9.153683e-03,5.609628e-07,1.540727e-04,7.136915e-09,1.368164e-03,1.370599e-06,7.105721e-05,1.372787e-03,1.526903e-03
1,1.511638e-07,6.271142e-06,2.036370e-03,9.038667e-04,1.018601e-02,3.003046e-06,4.659577e-05,2.116507e-06,1.129592e-07,1.728624e-05,...,7.205716e-04,1.075639e-03,4.814690e-07,1.739945e-05,3.767681e-08,1.567135e-06,2.052840e-04,2.052322e-02,2.748728e-05,3.273806e-07
2,8.772491e-13,3.620785e-08,8.009392e-04,1.627400e-04,1.170393e-01,1.445138e-11,1.792301e-07,1.092387e-09,8.538181e-07,2.417128e-08,...,2.031927e-04,1.541040e-07,3.156807e-08,1.824085e-03,5.448002e-09,3.594116e-04,4.366271e-08,5.042875e-06,8.282229e-04,1.213957e-04
3,1.010227e-06,3.042602e-07,9.576094e-05,7.613872e-06,8.445079e-04,1.622459e-04,2.218663e-07,5.444295e-10,1.512837e-08,2.928873e-07,...,5.053560e-08,4.297567e-02,1.222905e-06,2.360409e-04,2.946462e-12,2.158532e-07,3.490223e-09,1.202043e-03,2.716021e-03,9.927994e-06
4,3.192942e-05,2.220893e-05,2.757205e-08,5.218703e-07,8.056850e-09,1.248341e-06,2.633384e-11,2.303849e-07,2.973946e-08,9.021240e-07,...,4.730752e-06,1.954864e-08,7.331705e-07,6.909420e-09,8.526355e-08,1.816685e-12,7.951930e-05,1.952529e-11,1.267734e-07,6.001258e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4.220821e-04,1.486664e-05,5.663338e-09,2.856021e-04,1.105136e-06,2.435163e-12,9.642803e-04,5.645821e-06,8.174762e-08,1.495787e-05,...,3.973597e-09,1.754184e-07,1.572315e-03,1.281827e-04,9.474374e-14,1.399717e-09,3.313992e-05,1.678192e-07,5.898331e-07,7.323881e-05
9996,3.122185e-12,5.658266e-10,2.787805e-06,2.386513e-04,2.302973e-03,2.137102e-09,6.775127e-06,5.443222e-07,1.570331e-13,8.105628e-11,...,1.957740e-06,1.044687e-06,3.042427e-09,1.699808e-06,5.082691e-13,4.071443e-07,6.920533e-11,2.791522e-06,2.390137e-06,1.639048e-06
9997,3.389526e-06,1.025101e-06,5.978344e-08,7.857775e-05,2.096892e-06,2.839526e-06,2.631616e-02,6.525319e-04,3.136982e-07,5.859233e-07,...,1.129272e-03,4.489306e-04,2.854806e-02,1.122509e-05,1.665750e-11,1.605582e-07,6.704971e-05,1.408988e-03,4.915388e-06,3.812584e-02
9998,1.307755e-07,1.672701e-10,6.470808e-05,3.589738e-03,1.837592e-03,2.394744e-05,2.565926e-06,1.371135e-06,3.304201e-04,5.238783e-06,...,1.493052e-04,7.382702e-05,4.228802e-06,1.316629e-04,2.760068e-08,4.071098e-10,4.534271e-07,6.869188e-01,4.514720e-05,9.603621e-03


In [ ]:
results_16_b.to_csv(filepath +'25_softmax.csv',index =False)

# Download Data

In [ ]:
shutil.make_archive("quantised_16_bit", 'zip', "quantised_16_bit")
files.download("quantised_16_bit.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("quantised_8_bit", 'zip', "quantised_8_bit")
files.download("quantised_8_bit.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>